# YOLO Step 3 v.11

*withOUT Synthetic data*

Roboflow data generated:
* 2022-03-17 no_augmentation
* Version 14


In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import io
import pandas as pd
import json
import os
import glob 


In [3]:
roboflow_version = 14
parent_dir = "/home/ec2-user/AutoArki/yolov5/yolov5/autoarki-"

In [14]:
#!pip install roboflow
os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5")

from roboflow import Roboflow
rf = Roboflow(api_key="pvpFbwxA6FFS6pAwYdNk")
project = rf.workspace("new-workspace-s32a1").project("autoarki")
dataset = project.version(roboflow_version).download("yolov5")

roboflow_version_str = str(roboflow_version)+"/"

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to autoarki-14 in yolov5pytorch:: 100%|██████████| 5176/5176 [00:00<00:00, 10139.77it/s]


Manually rename autoarki folder or update the directory path below

In [20]:
os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5")

!rm -R train; mkdir train
!rm -R test; mkdir test
!rm -R valid; mkdir valid
!cp {parent_dir+roboflow_version_str}train/images/*  train --recursive
!cp {parent_dir+roboflow_version_str}train/labels/*  train --recursive
!cp {parent_dir+roboflow_version_str}test/images/*  test --recursive
!cp {parent_dir+roboflow_version_str}test/labels/*  test --recursive
!cp {parent_dir+roboflow_version_str}valid/images/*  valid --recursive
!cp {parent_dir+roboflow_version_str}valid/labels/*  valid --recursive



# Check 

In [21]:
def count_classes(file_path):
    '''
    increment class_counts dictionary for each instance of a 
    class check against roboflow summary (health check)
    
    '''
    infile = open(file_path, "r")
    lines = infile.readlines()

    if len(lines) == 0:
        pass
    else:
        for og_line in lines:
            class_counts[og_line[0]] +=1
    return class_counts

**Class Order**  <br>
0: '-' <br>
1: Column_Wall_Intersection <br>
2: cmu_hinge_clearance <br>
3: floating_door <br>
4: illegibletext <br>
5: missing_wall <br>
6: n <br>
7: no_stair_direction <br>

In [22]:
#Count of class types in train
class_counts = {'0':0, '1':0, '2':0, '3': 0, '4': 0, '5': 0, '6':0, '7':0}

os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/train")
for file in glob.glob("*.txt"):
    count_classes(file)
print(class_counts)

{'0': 1, '1': 306, '2': 21, '3': 552, '4': 4997, '5': 133, '6': 2, '7': 474}


# Replace Class Type

Drop -, cmu, and missing_wall classes

In [23]:
classes = {'1': '0', # Column Wall intersection
           '3': '1', # Floating Door
           '4': '2', # Illegible Text
           '7': '3'  # No stair direction
          } 

classes_to_omit = ['0','2','5','6']

In [24]:
def replace_class_n_with_class_name(file_path):
    '''for a txt file replace the class number with a class name'''
    new_lines = []
    infile = open(file_path, "r")
    lines = infile.readlines()

    if len(lines) == 0:
        pass
    else:
        for og_line in lines:
            class_n = og_line[0]
            if class_n in classes_to_omit: 
                continue
            class_str = classes[class_n]
            new_line = og_line.replace(class_n, class_str, 1)
            new_lines.append(new_line)

    with open(file_path, 'w') as outfile:
        outfile.write(''.join(new_lines))

In [25]:
# replace class names for test and validation txt files

os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/train")
for file in glob.glob("*.txt"):
    replace_class_n_with_class_name(file)
    
os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/valid")
for file in glob.glob("*.txt"):
    replace_class_n_with_class_name(file)    
    

# Install W&B login to track model - only do once

In [ ]:
!pip install wandb
!wandb login

Defaulting to user installation because normal site-packages is not writeable
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


In [40]:
import wandb

wandb.init(project="my-test-project", entity="kbaumstan")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /home/ec2-user/.netrc
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


# Learning

use smallest YOLO model to start yolov5s.pt

In [26]:
import torch
import torchvision

'0' # Column Wall intersection <br>
'1', # Floating Door <br>
'2', # Illegible Text <br>
'3'  # No stair direction <br>


## 50 Epochs run on best model weights from experiment YOLO Step 3- Train v.10

In [ ]:
# reduce epochs and set cos-lr to false
os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5")
!python train.py\
    --data custom_dataset.yaml \
    --epochs 300 \
    --project custom_yolov5 \
    --bbox_interval 1 \
    --save-period 10 \
    --img 1280 \
    --patience 10 \
    --weights best_exp39.pt \
    --optimizer 'AdamW' 
#    --cos-lr false\
#    --batch_size 10


wandb: Currently logged in as: auto-arki (use `wandb login --relogin` to force relogin)
train: weights=best_exp39.pt, cfg=, data=custom_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=16, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=AdamW, sync_bn=False, workers=8, project=custom_yolov5, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=10, freeze=[0], save_period=10, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 27 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-11-g63ddb6f torch 1.10.2+cu102 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.